In [1]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
folder_name = '成品采购明细/'
if not os.path.exists('成品采购明细'):
    os.makedirs('成品采购明细')


month_name = '成品采购明细'
input_filename = month_name +'.xls'
output_filename = folder_name + month_name+ '_添加类别.xlsx'

df = pd.read_excel(input_filename, sheetname ='成品采购明细1月')
df.columns.values

WARNING *** file size (1080650) not 512 + multiple of sector size (512)


array(['属性', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '入库统计',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', '出库统计', 'Unnamed: 33'], dtype=object)

In [2]:
# 去除 出库统计 金额
df.drop(['出库统计','Unnamed: 33'], axis=1, inplace = True)


In [3]:
names = df.loc[0,:]
names[names == '类别名称'].index


Index(['Unnamed: 8'], dtype='object')

In [4]:
df.rename(index=str, columns={'Unnamed: 8': 'type_name'},inplace=True)
df.type_name.unique()

array(['类别名称', 'XG', 'W', 'XF', 'XW', 'WM', 'HNJ', 'HDF', 'PG', '文具', '腰线',
       '浮雕', '花片', 'HNG', 'XT', '欧驰', 'HNT', 'HNF', 'HDJ', 'HDT', 'HM',
       'HTF', '纸箱', '古宝斯', 'HY', '其它费用', nan], dtype=object)

In [5]:
df.rename(index=str, columns={'Unnamed: 9': 'pd_size'}, inplace = True)

In [6]:
df.insert(8,'type',df.type_name)
df.columns.values

array(['属性', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'type', 'type_name',
       'pd_size', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '入库统计',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31'], dtype=object)

In [7]:
df.type[df.type_name == '类别名称'] = '类别'

df.type[(df.type_name =='HDF') | (df.type_name == 'HDG') | (df.type_name == 'HDJ') | (df.type == 'HDT') ] = '地砖'
df.type[df.type_name =='HM'] = '木纹砖'
df.type[(df.type_name =='HNF') | (df.type_name == 'HNG') | (df.type_name == 'HNJ') | (df.type == 'HNT') ] = '瓷片'
df.type[df.type_name =='HTF'] = '地砖'
df.type[df.type_name == 'HW'] = '微晶'
df.type[df.type_name == 'HY'] = '全抛釉'

df.type[df.type_name == 'PG'] ='通体砖'
df.type[(df.type_name == 'PG') & (df.pd_size == '300X300')] = '地砖'
df.type[(df.type_name == 'PG') & (df.pd_size == '300X600')] = '瓷片'

df.type[(df.type_name == 'P') | (df.type_name == 'T')] = '通体砖'
df.type[(df.type_name == 'W') | (df.type_name == 'WM')] = '瓷片'
df.type[df.type_name == 'XF'] = '地砖'
df.type[(df.type_name == 'XG') | (df.type_name == 'XT') | (df.type_name == 'XW')] = '瓷片'
df.type[df.type_name == 'H3S'] = '色晶砖'
df.type[df.type_name == 'HS'] = '大理石'
df.type[df.type_name == '浮雕'] = '浮雕'
df.type[df.type_name == '文具'] = '五金仓'
df.type[df.type_name == '花片'] = '花片'
df.type[df.type_name == '欧驰'] = '欧驰'
df.type[df.type_name == '腰线'] = '腰线'
df.type[df.type_name == '纸箱'] = '纸箱'

In [8]:
print(df.groupby('type').type.agg('count'))

type
五金仓       24
全抛釉        4
其它费用       2
古宝斯        4
地砖       187
木纹砖        3
欧驰        13
浮雕         3
瓷片      1231
类别         1
纸箱         1
腰线        20
花片        39
通体砖       12
Name: type, dtype: int64


In [9]:
#modified output
output_df = df.copy(deep=True)
output_df.columns = output_df.iloc[0,:]
output_df.drop(['0'], axis=0,inplace = True)
output_df.set_index('单号',inplace = True)



writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
output_df.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

In [10]:
df2 = output_df.copy(deep=True)
df2.replace(np.nan,'',regex = True, inplace = True)
output_file_list = list(df2['供应商名'].unique())
output_file_list

['广西恒希建材有限公司',
 '佛山市博今科技材料有限公司',
 '澳琪腰线厂B（发外加工）',
 '欧驰建材有限公司',
 '湖兴仓',
 '外购',
 '古宝斯',
 '新顺兴陶瓷有限公司',
 '百顺达陶瓷有限公司',
 '壹号工匠瓷砖胶',
 '']

In [33]:

for key, sub_df in df2.groupby('供应商名'):
    if key != '':
        print(key)
        sub_df = sub_df.apply(pd.to_numeric,errors='ignore')
        grouped = pd.concat([ sub_df.groupby('类别')['金额'].agg(np.sum),
                              sub_df.groupby('类别')['加工金额'].agg(np.sum),
                              sub_df.groupby('类别')['装车金额'].agg(np.sum),
                              sub_df.groupby('类别')['纸箱金额'].agg(np.sum),
                              sub_df.groupby('类别')['总金额'].agg(np.sum)],
                            axis = 1)
        
        grouped.loc[grouped.shape[0]] = grouped.sum()
        grouped.rename(index = {grouped.shape[0]-1: '总计'}, inplace = True)
        
        output_filename = folder_name+'/summary_'+key+'.xlsx'
        writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
        grouped.to_excel(writer,'sheet1') #creates workbook 
        writer.save() #saves workbook to file in python file directory
    

佛山市博今科技材料有限公司
古宝斯
壹号工匠瓷砖胶
外购
广西恒希建材有限公司
新顺兴陶瓷有限公司
欧驰建材有限公司
湖兴仓
澳琪腰线厂B（发外加工）
百顺达陶瓷有限公司
